## 5.3 多输入通道和多输出通道

- 假设彩色图像的高和宽分别是$h$和$w$(像素)，那么它可以表示为一个$3\times h\times w$的多维数组。
- 我们将大小为3的这一维称为通道(channel)维。

### 5.3.1 多输入通道

- 当输入数据含多个通道时，我们需要构造一个输入通道数与输入数据的通道数相同的卷积核，从而能够与含多通道的输入数据做互相关运算。
- 假设输入数据的通道数为$c_i$，那么卷积核的输入通道数同样为$c_i$。
- 把这$c_i$个数组在输入通道维上连结，即得到一个形状为$c_i\times k_h\times k_w$的卷积核。

In [1]:
import d2lzh as d2l
from mxnet import nd

def corr2d_multi_in(X, K):
    # 首先沿着X和K的第0维(通道维)遍历。然后使用*将结果列表变成add_n函数的位置参数来进行相加
    return nd.add_n(*[d2l.corr2d(x, k) for x, k in zip(X, K)])

In [2]:
X = nd.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]],
              [[1, 2, 3], [4, 5, 6], [7, 8, 9]]])
K = nd.array([[[0, 1], [2, 3]], [[1, 2], [3, 4]]])

corr2d_multi_in(X, K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

### 5.3.2 多输出通道

- 当输入通道数有多个时，因为我们对各个通道的结果做了累加，所以不论输入通道数是多少，输出通道数总是为1.
- 设卷积核输入通道数和输出通道数分别为$c_i$和$c_o$，高和宽分别为$k_h$和$k_w$。
- 如果希望得到含多个通道的输出，我们可以为每个输出通道分别创建形状为$c_i\times k_h\times k_w$的核数组。将它们在输出通道维上连结，卷积核的形状即$c_o\times c_i\times k_h\times k_w$。
- 在做互相关运算时，每个输出通道上的结果由卷积核在该通道上的核数组与整个输入数组计算而来。

In [4]:
def corr2d_multi_in_out(X, K):
    # 对K的第0维遍历，每次同输入X做互相关运算。所有结果使用stack函数合并在一起
    return nd.stack(*[corr2d_multi_in(X, k) for k in K])

In [5]:
K = nd.stack(K, K + 1, K + 2)
K.shape

(3, 2, 2, 2)

In [6]:
corr2d_multi_in_out(X, K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

### 5.3.3  $1\times 1$卷积层

- 我们通常称之为$1\times 1$卷积层，并将其中的卷积运算称为$1\times 1$卷积。
- 因为使用了最小窗口，$1\times 1$卷积失去了卷积层可以识别高和宽维度上相邻元素构成的模式的功能。
- 输出中的每个元素来自输入中在高和宽上相同位置的元素在不同通道之间的按权重累加。
- 假设我们将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    Y = nd.dot(K, X) # 全连接层的矩阵乘法
    return Y.reshape((c_o, h, w))

In [8]:
X = nd.random_uniform(shape=(3, 3, 3))
K = nd.random_uniform(shape=(2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)

Y1, Y2

(
 [[[0.63251853 0.7027089  0.8923846 ]
   [1.0109798  0.70354044 0.9426006 ]
   [0.65125    0.909348   0.68587   ]]
 
  [[0.45974678 0.47996217 0.3274668 ]
   [0.3736128  0.2560653  0.66247874]
   [0.09572    0.7063811  0.23235497]]]
 <NDArray 2x3x3 @cpu(0)>,
 
 [[[0.63251853 0.70270896 0.8923846 ]
   [1.0109798  0.70354044 0.9426006 ]
   [0.65125    0.909348   0.68587   ]]
 
  [[0.45974678 0.47996217 0.3274668 ]
   [0.37361276 0.2560653  0.66247874]
   [0.09572    0.7063811  0.23235497]]]
 <NDArray 2x3x3 @cpu(0)>)

- $1\times 1$卷积层被当作保持高和宽维度形状不变的全连接层使用。
- 我们可以通过调整网络层之间的通道数来控制模型复杂度。

### 小结

- 使用多通道可以拓展卷积层的模型参数。
- 假设将通道维当作特征维，将高和宽维度上的元素当成数据样本，那么$1\times 1$卷积层的作用与全连接层等价。
- $1\times 1$卷积层通常用来调整网络层之间的通道数，并控制模型复杂度。